#AirSimSol

In [ ]:
import gym
import time
the same thing but with ppo algorithim import numpy as np
import tensorflow as tf
import gym
import random
import copy
import math

import roslib
import rospy
import random
import time
import math
from std_srvs.srv import Empty as empty
from std_msgs.msg import Empty
from gazebo_msgs.srv import SetModelConfiguration

from control_msgs.msg import JointControllerState
from sensor_msgs.msg import JointState
from gazebo_msgs.msg import LinkStates
from std_msgs.msg import Float64
from std_msgs.msg import String
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from ardrone_autonomy.msg import Navdata
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage
from stable_baselines3.common.callbacks import EvalCallback


def main():
    env = DummyVecEnv(
        [
            lambda: Monitor(
                gym.make(
                    "CartPole-v1",  # Use a predefined gym environment like CartPole
                )
            )
        ]
    )

    model = PPO(
        "MlpPolicy",  # MlpPolicy is generally used for environments like CartPole
        env,
        learning_rate=0.00025,
        verbose=2,
        batch_size=512,
        max_grad_norm=0.5,
        device="cuda",
        tensorboard_log=None,
    )

    callbacks = []
    eval_callback = EvalCallback(
        env,
        callback_on_new_best=None,
        n_eval_episodes=5,
        best_model_save_path=".",
        log_path=".",
        eval_freq=10000,
    )
    callbacks.append(eval_callback)

    model.learn(
        total_timesteps=int(5e5),
        tb_log_name="cartpole_run_" + str(time.time()),
        callback=callbacks
    )

    # Save policy weights
    model.save("cartpole_policy")


if __name__ == '__main__':
    main()
